In [ ]:
# default_exp proxy

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
!nbdev_build_lib --fname 11_Proxy_Request.ipynb

Converted 11_Proxy_Request.ipynb.


# TODO
- [x] 2020-03-15 抽象成Class

# 代理爬虫
> 很多网站都有反爬虫机制，一个IP频繁访问一个网站，就会出现访问被拒绝的情况，所以换IP可以解决这个问题。（运用技术请克制，避免过度浪费服务器资源）

## 测试代理
> 有专用的付费代理IP稳定可靠，也有免费的代理IP可能会随时失效

In [ ]:
# export
import requests,json,re,random,sys,time,os
from bs4 import BeautifulSoup,Tag,NavigableString

from crawler_from_scratch.utils import *

from concurrent.futures import ThreadPoolExecutor
import pandas as pd


先从 https://www.freeip.top/ 随便拿个ip来测试

In [ ]:
# hide
url = 'https://www.baidu.com/'
headers={'user-agent':'Mozilla/5.0'}
proxies = {'https': 'https://64.227.1.188:8080'}
res = requests.get(url,proxies=proxies,headers=headers,timeout=5)
res

<Response [200]>

## 制作代理池
> 网上有专门整理的[代理池](https://github.com/jhao104/proxy_pool)，但需要配置数据库，所以不在这里演示，而是自己写个爬虫，通过[这个网站的API](https://github.com/jiangxianli/ProxyIpLib)获取IP

In [ ]:
# export
class Proxy():
    '一个代理器，用爬取的免费代理ip，来爬取网站'
    def __init__(self):
        self.db = {}
        self.path = './data/11_Proxy.json'
        self.debug = True
        
        if os.path.exists(self.path):
            with open(self.path, 'r') as f:
                self.db = json.loads(f.read())
                print('加载成功',len(self.db.keys()))
        else:
            self.update(self)
    
    def update(self):
        '重新爬取ip，初始化health值，赋值给`self.db`，保存在`self.path`'
        data = []
        next_page_url = 'https://www.freeip.top/api/proxy_ips?page=1'
        while next_page_url:
            if self.debug: print('start:',next_page_url)
            res = requests.get(next_page_url)
            if res.status_code == 200:
                data_list = res.json()['data']['data']   
                data += data_list
                next_page_url = res.json()['data']['next_page_url']
            time.sleep(1)
        # ip list 转 dict 增加健康值
        self.db = {}
        for d in data:
            _id = d['unique_id']
            self.db[_id] = d
            self.db[_id]['health'] = 50
        
        self.save()
            
        self.validate('http://www.baidu.com/')
        self.validate('https://www.baidu.com/')
    
    def save(self):    
        with open(self.path, 'w') as f:
            json.dump(self.db,f)
            print('更新成功',len(self.db.keys()))
    
    def validate(self,url,max_workers=50):
        '批量测试ip有效性'
        protocol = url.split(':')[0]
        db_with_protocol = [self.db[k] for k in self.db if self.db[k]['protocol'] == protocol]
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            executor.map(lambda ip_obj : self._get(url,ip_obj),db_with_protocol)
        
    def choose_healthy_ip(self,protocol):
        '根据健康度，随机选择优质ip'
        db_with_protocol = [self.db[k] for k in self.db if self.db[k]['protocol'] == protocol]
        sorted_db = sorted(db_with_protocol, 
                           key = lambda item : item['health'],
                           reverse=True)
        return random.choice(sorted_db[:10])
        
    def update_ip_health(self,res,obj):
        '根据response，更新health'
        if res.status_code == 200:
            obj['health'] += 1
        else:
            obj['health'] = int(obj['health']/2)
        if self.debug: print(obj['ip'],'健康值变为：',obj['health'])
        
    def _get(self,url,ip_obj={}):
        '如果不指定ip，则自动选择`self.db`中最优的ip，访问网页，并更新health值'
        protocol = url.split(':')[0]
        if not ip_obj: ip_obj = self.choose_healthy_ip(protocol)
        ip = f"{ip_obj['protocol']}://{ip_obj['ip']}:{ip_obj['port']}"
        
        try:
            res = requests.get(url,
                               proxies={protocol: ip},
                               headers={'user-agent':'Mozilla/5.0'},
                               timeout=5)
        except:
            if self.debug: print(f'error: {ip}\n{sys.exc_info()}\n')
            res = requests.Response()
        
        self.update_ip_health(res,ip_obj)
            
        return res
    def get(self,url):
        '如果一个网页访问失败，会更换ip重试10次'
        try_times = 1
        while try_times < 11:
            if self.debug : print('\n',try_times,url)
            res = self._get(url)
            if res.status_code == 200:
                print('访问成功：',url)
                return res
            else:
                try_times += 1
        print('访问失败：',url)
        return res
    


In [ ]:
# hide
px = Proxy()
px.update()

加载成功 365
start: https://www.freeip.top/api/proxy_ips?page=1
start: https://www.freeip.top/api/proxy_ips?page=2
start: https://www.freeip.top/api/proxy_ips?page=3
start: https://www.freeip.top/api/proxy_ips?page=4
start: https://www.freeip.top/api/proxy_ips?page=5
start: https://www.freeip.top/api/proxy_ips?page=6
start: https://www.freeip.top/api/proxy_ips?page=7
start: https://www.freeip.top/api/proxy_ips?page=8
start: https://www.freeip.top/api/proxy_ips?page=9
start: https://www.freeip.top/api/proxy_ips?page=10
start: https://www.freeip.top/api/proxy_ips?page=11
start: https://www.freeip.top/api/proxy_ips?page=12
start: https://www.freeip.top/api/proxy_ips?page=13
start: https://www.freeip.top/api/proxy_ips?page=14
start: https://www.freeip.top/api/proxy_ips?page=15
start: https://www.freeip.top/api/proxy_ips?page=16
start: https://www.freeip.top/api/proxy_ips?page=17
start: https://www.freeip.top/api/proxy_ips?page=18
start: https://www.freeip.top/api/proxy_ips?page=19
start: https

183.232.232.69 健康值变为： 51
163.172.146.119 健康值变为： 51
183.56.161.62 健康值变为： 51
167.71.197.226 健康值变为： 51
159.203.166.41 健康值变为： 51
error: http://128.199.245.21:44344
(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='128.199.245.21', port=44344): Read timed out.")), <traceback object at 0x120a81370>)

128.199.245.21 健康值变为： 25
178.128.87.98 健康值变为： 51
178.128.126.135 健康值变为： 51
128.199.177.120 健康值变为： 51
185.80.128.166 健康值变为： 51
157.230.241.171 健康值变为： 51
159.203.164.91 健康值变为： 51
176.53.40.222 健康值变为： 51
138.197.133.199 健康值变为： 51
151.253.165.70 健康值变为： 51
178.128.16.115 健康值变为： 51
103.235.46.121 健康值变为： 51
183.146.213.198 健康值变为： 51
error: http://121.67.3.3:8080
(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='121.67.3.3', port=8080): Read timed out.")), <traceback object at 0x10ddee050>)

121.67.3.3 健康值变为： 25
128.199.237.185 健康值变为： 51
218.75.158.153 健康值变为： 51
error: http://167.99.185.216:

51.79.85.125 健康值变为： 51
68.183.178.107 健康值变为： 51
error: http://60.51.170.27:80
(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='60.51.170.27', port=80): Read timed out.")), <traceback object at 0x10de3a5a0>)

60.51.170.27 健康值变为： 25
85.95.220.32 健康值变为： 51
error: http://190.103.178.14:8080
(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='190.103.178.14', port=8080): Read timed out.")), <traceback object at 0x120cecf00>)

190.103.178.14 健康值变为： 25
68.183.237.110 健康值变为： 51
84.17.47.187 健康值变为： 51
84.17.47.183 健康值变为： 51
84.17.47.190 健康值变为： 51
error: http://84.17.47.191:80
(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ReadTimeoutError("HTTPConnectionPool(host='84.17.47.191', port=80): Read timed out.")), <traceback object at 0x120cd8d70>)

84.17.47.191 健康值变为： 25
200.89.178.210 健康值变为： 51
84.17.47.184 健康值变为： 51
78.166.75.176 健康值变为： 51
84.17.47.182 健康值变为： 51
84.17.4

178.128.31.220 健康值变为： 51
58.240.97.154 健康值变为： 51
209.97.183.194 健康值变为： 51
52.80.58.248 健康值变为： 51
178.128.221.73 健康值变为： 51
5.44.107.147 健康值变为： 51
51.158.114.177 健康值变为： 51
51.158.108.135 健康值变为： 51
error: https://206.189.154.176:8080
(<class 'requests.exceptions.ProxyError'>, ProxyError(MaxRetryError("HTTPSConnectionPool(host='www.baidu.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out')))")), <traceback object at 0x1209a19b0>)

206.189.154.176 健康值变为： 25
52.140.242.103 健康值变为： 51
176.53.40.222 健康值变为： 51
178.128.87.184 健康值变为： 51
128.199.66.13 健康值变为： 51
200.73.128.63 健康值变为： 51
24.113.36.247 健康值变为： 51
51.158.99.51 健康值变为： 51
51.158.165.18 健康值变为： 51
51.158.123.250 健康值变为： 51
error: https://46.28.95.11:3128
(<class 'requests.exceptions.ConnectTimeout'>, ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='www.baidu.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTT

## 自动切换代理
> 优先选择健康值高的ip，一次请求，成功健康值+1，失败则减半

In [ ]:
# hide
px = Proxy()
px._get('https://www.baidu.com/')

加载成功 364
118.70.144.77 健康值变为： 51


<Response [200]>

## 校验&更替代理
> 用百度批量测试网站的有效性

In [ ]:
# hide
px = Proxy()
px.validate('http://www.baidu.com/')

## 用代理爬取豆瓣页面
> 昨天用豆瓣页面测试爬虫功能的时候，就发现了访问频率过高的被拒的问题，今天就用爬虫来抓取整个互联网类目下的图书信息

In [ ]:
url_list = []
for i in range(0,1000,20):
    url = f'https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start={i}&type=T'
    url_list.append(url)
len(url_list)

50

In [ ]:
def get_douban_data(url,px,data):
    res = px.get(url)    

    if res.status_code == 200:
        soup = BeautifulSoup(res.text)
        main_content = soup.body.find('ul',class_='subject-list')

        for c in get_children(main_content):
            item_data = get_data(c)
            a_nbg_url = item_data['a_nbg_url']
            _id = re.search(r'/(\d+)/',a_nbg_url).group(1)
            # 写入data
            data[_id] = item_data
    else:
        print(res,res.text)

In [ ]:
# hide
px = Proxy()
data ={}
px.debug = True

for i in range(3):
    px.validate('https://book.douban.com')   

px.save()


In [ ]:
#hide
px.debug = False
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(lambda url : get_douban_data(url,px,data), url_list) 


访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=180&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=120&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=40&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=20&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=260&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=0&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=160&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=60&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=300&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=200&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=220&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=80&type=T
访问成功： https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD

In [ ]:
# hide
dataframe = pd.DataFrame.from_dict(data,orient='index')
dataframe.head()

,a_nbg_url,img_no_class_src,a_no_class_url,a_no_class_title,a_no_class_text,span_no_class_text,div_pub_text,span_rating_nums_text,span_pl_text,p_no_class_text
3191237,https://book.douban.com/subject/3191237/,https://img3.doubanio.com/view/subject/s/publi...,https://book.douban.com/subject/3191237/buylinks,众声喧哗,纸质版 18.60 元起,: 网络时代的个人表达与公共讨论,胡泳 / 广西师范大学出版社 / 2008-9 / 35.00元,8.0,(907人评价),本书触及了网络政治学中的一个重大话题——网络空间中的私域与公域。随着科技的进步，在信息时代的...
30364484,https://book.douban.com/subject/30364484/,https://img1.doubanio.com/view/subject/s/publi...,https://read.douban.com/ebook/60171085/?dcs=ta...,创投42章经,去看电子版,: 互联网商业逻辑与投资进阶指南,曲凯 / 中信出版集团 / 2018-10-20 / 58.00,8.2,(212人评价),《创投42章经》是拥有百万粉丝的微信公众号“42章经”的精选文章合集，全书共分为心法、内功、...
25843241,https://book.douban.com/subject/25843241/,https://img3.doubanio.com/view/subject/s/publi...,https://book.douban.com/subject/25843241/buylinks,互联网思维独孤九剑,纸质版 37.40 元起,: 移动互联时代的思维革命,赵大伟 / 机械工业出版社 / 2014-3-20 / 49,7.3,(842人评价),《互联网思维独孤九剑》是国内第一部系统阐述互联网思维的著作，用9大互联网思维：用户思维、简约...
26400900,https://book.douban.com/subject/26400900/,https://img3.doubanio.com/view/subject/s/publi...,https://read.douban.com/ebook/25462377/?dcs=ta...,创京东,去看电子版,: 刘强东亲述创业之路,李志刚 / 中信出版社 / 2015-5-1 / CNY 49.80,7.1,(2242人评价),1998年，刘强东创业，在中关村经销光磁产品。2004年，因为非典，京东偶然之下转向线上销售...
20388034,https://book.douban.com/subject/20388034/,https://img3.doubanio.com/view/subject/s/publi...,https://book.douban.com/subject/20388034/buylinks,大连接,纸质版 44.90 元起,: 社会网络是如何形成的以及对人类现实行为的影响,[美] 尼古拉斯•克里斯塔基斯（Nicholas A. Christakis）、[美] 詹姆...,7.2,(561人评价),[内容简介]\n1. 本书是继《六度分隔》之后，社会科学领域最重要的作品。作者发现：相距三度...


得到了这样的规范的数据结构的数据，就可以进行很多数据分析的工作，比如评价数分布，评分分布，热门作者等等

In [ ]:
# hide
dataframe.describe()

,a_nbg_url,img_no_class_src,a_no_class_url,a_no_class_title,a_no_class_text,span_no_class_text,div_pub_text,span_rating_nums_text,span_pl_text,p_no_class_text
count,1000,1000,1000,999,1000,573,997,921,1000,945
unique,1000,999,1000,957,379,561,997,52,572,938
top,https://book.douban.com/subject/26177913/,https://img9.doubanio.com/f/shire/5522dd1f5b74...,https://book.douban.com/subject/24875857/buylinks,引爆点,去看电子版,: 无组织的组织力量,李彦 / 清华大学 / 2005-7 / 66.00元,8.0,(少于10人评价),计算广告是一项新兴的研究课题，它涉及大规模搜索和文本分析、信息获取、统计模型、机器学习、分类...
freq,1,2,1,3,312,3,1,51,66,2
